# Summary:
- 1D Cov + two layers of LSTM is tested


In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [5]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [6]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((7662, 1024), (7662,), (2554, 1024), (2554,), 10216)

In [7]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [8]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],32,32) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],32,32) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [9]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [10]:
model = model(input_shape = (32,32))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 768)            369408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 768)            3072      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 768)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 768)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 512)            2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
__________

In [12]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [13]:
model.fit(X_train, Y_train, batch_size = 64, epochs=200)

Epoch 1/200
7662/7662 [==============================] - 8s 1ms/step - loss: 0.4392 - acc: 0.8403
Epoch 2/200
7662/7662 [==============================] - 5s 618us/step - loss: 0.4110 - acc: 0.8427
Epoch 3/200
7662/7662 [==============================] - 5s 598us/step - loss: 0.3896 - acc: 0.8460
Epoch 4/200
7662/7662 [==============================] - 5s 612us/step - loss: 0.3732 - acc: 0.8512
Epoch 5/200
7662/7662 [==============================] - 5s 608us/step - loss: 0.3620 - acc: 0.8526
Epoch 6/200
7662/7662 [==============================] - 5s 609us/step - loss: 0.3454 - acc: 0.8603
Epoch 7/200
7662/7662 [==============================] - 5s 606us/step - loss: 0.3297 - acc: 0.8613
Epoch 8/200
7662/7662 [==============================] - 5s 602us/step - loss: 0.3082 - acc: 0.8739
Epoch 9/200
7662/7662 [==============================] - 5s 603us/step - loss: 0.2908 - acc: 0.8835
Epoch 10/200
7662/7662 [==============================] - 5s 602us/step - loss: 0.2889 - acc: 0.8812
E

7662/7662 [==============================] - 5s 592us/step - loss: 0.1185 - acc: 0.9521
Epoch 83/200
7662/7662 [==============================] - 5s 589us/step - loss: 0.1208 - acc: 0.9526
Epoch 84/200
7662/7662 [==============================] - 5s 595us/step - loss: 0.1210 - acc: 0.9501
Epoch 85/200
7662/7662 [==============================] - 5s 590us/step - loss: 0.1225 - acc: 0.9514
Epoch 86/200
7662/7662 [==============================] - 5s 594us/step - loss: 0.1126 - acc: 0.9560
Epoch 87/200
7662/7662 [==============================] - 5s 590us/step - loss: 0.1241 - acc: 0.9511
Epoch 88/200
7662/7662 [==============================] - 5s 593us/step - loss: 0.1164 - acc: 0.9528
Epoch 89/200
7662/7662 [==============================] - 5s 591us/step - loss: 0.1185 - acc: 0.9529
Epoch 90/200
7662/7662 [==============================] - 5s 591us/step - loss: 0.1169 - acc: 0.9525
Epoch 91/200
7662/7662 [==============================] - 5s 594us/step - loss: 0.1205 - acc: 0.9529
Epo

7662/7662 [==============================] - 5s 594us/step - loss: 0.0854 - acc: 0.9679
Epoch 163/200
7662/7662 [==============================] - 5s 588us/step - loss: 0.0953 - acc: 0.9632
Epoch 164/200
7662/7662 [==============================] - 5s 593us/step - loss: 0.0964 - acc: 0.9633
Epoch 165/200
7662/7662 [==============================] - 5s 589us/step - loss: 0.0966 - acc: 0.9632
Epoch 166/200
7662/7662 [==============================] - 5s 594us/step - loss: 0.0917 - acc: 0.9649
Epoch 167/200
7662/7662 [==============================] - 5s 589us/step - loss: 0.0944 - acc: 0.9640
Epoch 168/200
7662/7662 [==============================] - 5s 592us/step - loss: 0.0929 - acc: 0.9655
Epoch 169/200
7662/7662 [==============================] - 5s 588us/step - loss: 0.0941 - acc: 0.9650
Epoch 170/200
7662/7662 [==============================] - 5s 595us/step - loss: 0.0936 - acc: 0.9629
Epoch 171/200
7662/7662 [==============================] - 5s 593us/step - loss: 0.0977 - acc: 0

In [14]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

2554/2554 [==============================] - 1s 353us/step
Validation set accuracy =  0.835943617667644


In [15]:
model_json = model.to_json()
with open ("LSTMModelNoSMOTE.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModelNoSMOTE.h5")

In [16]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

2554/2554 [==============================] - 1s 312us/step
             precision    recall  f1-score   support

          0       0.52      0.06      0.10       421
          1       0.84      0.99      0.91      2133

avg / total       0.79      0.84      0.78      2554

